In [1]:
#%pip install tensorflow
#%pip install matplotlib
#%pip install pillow
#%pip install scipy
#%pip install opencv-python
#%pip install seaborn
#%pip install librosa
#%pip install pyaudio
#%pip install wave
#%pip install -U scikit-learn

In [3]:
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

[]


In [4]:
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
import numpy as np
import matplotlib as mpl
import cv2
import IPython.display as display
import PIL.Image
import time
import datetime as dt
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import pyaudio
import wave
from sklearn.preprocessing import StandardScaler, LabelEncoder
import speech_recognition as sr
import librosa
import subprocess
import json
import datetime
import wikipedia
import webbrowser
import ctypes
import requests
import pyttsx3
import openai
from urllib.request import urlopen
from joblib import dump, load
import urllib.request
import cv2
import numpy as np
import ssl
import tensorflow as tf
from tensorflow.keras.layers import Conv2D ,LSTM ,MaxPool2D , Input , GlobalAveragePooling2D ,AveragePooling2D, Dense , Dropout ,Activation, Flatten , BatchNormalization
import time
import keyboard 
import threading
import requests
import importlib
from statistics import mode
import mediapipe as mp
from mediapipe.python.solutions.drawing_utils import _normalized_to_pixel_coordinates



In [5]:
target_sample_rate = 44100  # Adjust the target sample rate as needed
interest = 'sports'
name = 'Hala'
messages = [{"role": "system", "content": "Hello! I am Ghost, a chatbot designed to keep the driver awake by engaging in conversation. Please keep your responses short to avoid distractions by talking about his interests the user's interest is "+interest+". If you wish to end the conversation, just say 'exit' or 'thank you'. the user name is " + name + ". Let's start by greeting you."}]

#executor=concurrent.futures.ThreadPoolExecutor(max_workers=10)
dist_predicted_class=''
drow_predicted_class=''

In [6]:
distraction_flag=0
drowsiness_flag=0
check_flag_drow=0
check_flag_dist=0

## Load models

In [7]:
Distraction_vision_model = tf.keras.models.load_model("C:/Users/lenovo/Desktop/Graduation_Project/Distraction/inceptionv3c_model.h5",compile=False)  # Load the inception distraction model
Speaking_voice_model = tf.keras.models.load_model("C:/Users/lenovo/Desktop/Graduation_Project/Sound/speaking_lstm_model2.h5",compile=False)
speaking_scalar=load('C:/Users/lenovo/Desktop/Graduation_Project/Sound/speaking_scaler.bin')
Drowsiness_vision_model =tf.keras.models.load_model("C:/Users/lenovo/Desktop/Graduation_Project/drowsiness/inceptionv3_model_drowsiness.h5",compile=False)
drowsiness_voice_model = tf.keras.models.load_model("C:/Users/lenovo/Desktop/Graduation_Project/Sound/drowsiness_sound_model.h5",compile=False)
drowsiness_scaler = load("C:/Users/lenovo/Desktop/Graduation_Project/Sound/drowsiness_sound_scaler.save")
Distraction_vision_lstm_model=tf.keras.models.load_model("C:/Users/lenovo/Desktop/Graduation_Project/Distraction/inceptionv3_lstm_model.h5")

c:\Users\lenovo\miniconda3\envs\gp\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.0.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\lenovo\miniconda3\envs\gp\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## Model functions for Camera Detection

In [8]:
def distraction_detection(img,class_names):
    resized_image=cv2.resize(img, (299, 299))
    # Preprocess the image for InceptionV3
    preprocessed_image = preprocess_input(resized_image)
    # Expand dimensions to create a batch (required for prediction)
    input_data = np.expand_dims(preprocessed_image, axis=0)
    # Make predictions (if needed)
    predictions = Distraction_vision_model.predict(input_data,verbose = 0)
    # Get the predicted class
    predicted_class_idx = np.argmax(predictions)
    predicted_class = class_names[predicted_class_idx]
    return resized_image,predicted_class


In [9]:
def distraction_lstm_detection(frames,class_names):
    images=[]
    for i in range(0,4):
        resized_image=cv2.resize(frames[i], (299, 299))
        # Preprocess the image for InceptionV3
        preprocessed_image = preprocess_input(resized_image)
        # Expand dimensions to create a batch (required for prediction)
        images.append(preprocessed_image)

    input_data = np.expand_dims(images, axis=0)
    # Make predictions (if needed)
    predictions = Distraction_vision_lstm_model.predict(input_data,verbose = 0)
    # Get the predicted class
    predicted_class_idx = np.argmax(predictions)
    predicted_class = class_names[predicted_class_idx]
    return predicted_class

In [10]:
def drowsiness_detection(eye):

    # Resize the eye to match the input shape of the model
    eye_resized = cv2.resize(eye, (80, 80))
    
    # Convert the eye to RGB
    eye_rgb = cv2.cvtColor(eye_resized, cv2.COLOR_BGR2RGB)
    preprocessed_image = preprocess_input(eye_rgb)
    
    # Expand the dimensions to create a batch of size 1
    input_eye = np.expand_dims(preprocessed_image, axis=0)
    
    # Perform prediction using the model
    prediction = Drowsiness_vision_model.predict(input_eye,verbose=0)
    return prediction

In [11]:
def open_file(filename,audio):
    with open(filename , "wb") as f:
        f.write(audio.get_wav_data())
    print("Opening wave file....")


## Real-Time Detection with Audio for Speaking 

In [12]:

def TakeRealTimeAudio():
	# threading.Thread(target=Real_Time_Recording).start()
    #Real_Time_Recording()
	filename = "C:/Users/lenovo/Desktop/Graduation_Project/sound/speaking_audio.wav" #the recorded audio will be saved there
	r = sr.Recognizer()
	source=sr.Microphone()
	#print(source.list_microphone_names())
	#lists =source.list_microphone_names()
	#index = lists.index('Microphone Array (Realtek(R) Audio)')
 	#device_index=index
	with sr.Microphone() as source:
		
		print("Listening...")
        
		r.pause_threshold = 1
		audio = r.listen(source=source, phrase_time_limit=6)
		print(type(audio))
		open_file(filename,audio)
		

	return filename

In [13]:

# Function to extract MFCC features from audio file
def extract_mfcc_file(file_path, target_sr=22050, n_mfcc=40):
    audio, sr = librosa.load(file_path, sr=target_sr, mono=True)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    # Fix the length if necessary
    mfccs = librosa.util.fix_length(data=mfccs, size=700, axis=1)
    return mfccs

def detect_speaking_file(model, audio_file_path, target_sr=22050, max_pad_len=700, gain_factor=10000.0):
    try:
        # Extract MFCC features from the audio file
        mfccs = extract_mfcc_file(audio_file_path, target_sr=target_sr)
        # scaler = StandardScaler()
        scaler=speaking_scalar
        # Scale the extracted features using the same scaler
        mfccs_scaled = scaler.transform(mfccs.reshape(1, -1)).reshape(mfccs.shape)

        # Reshape for model input
        input_data = np.expand_dims(mfccs_scaled, axis=0)

        # Make a prediction
        prediction = model.predict(input_data,verbose=0)

        # Interpret the prediction (you may need to adjust this based on your model output)
        print("Speaking Model in progress...")
        if prediction[0, 0] > 0.5:
            return"Speaking"
        else:
            return"Silent"

    except KeyboardInterrupt:
        print("Stopping detection.")



## Chatbot & Actions & Drowsiness by Voice Merged

In [14]:
import pyaudio
import wave
import numpy as np
import speech_recognition as sr
def extract_text_from_voice():
    # Initialize PyAudio
    p = pyaudio.PyAudio()

    # Define parameters for recording
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100
    CHUNK = 1024
    DURATION = 7  # Duration of recording is 7 seconds
    filename = "C:/Users/lenovo/Desktop/Graduation_Project/output.wav"

    # Open PyAudio stream for recording
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print('Listening ...')

    # Record the audio with increased volume and normalization
    frames = []
    gain_factor = 2.0  # Adjust the gain factor as needed
    for i in range(0, int(RATE / CHUNK * DURATION)):
        data = stream.read(CHUNK)
        audio_array = np.frombuffer(data, dtype=np.int16)

        # Apply gain and normalization
        scaled_audio = (audio_array * gain_factor).astype(np.int16)
        normalized_audio = np.int16(scaled_audio / np.max(np.abs(scaled_audio)) * 32767)

        frames.append(normalized_audio.tobytes())

    print('Stopped')

    # Stop and close the PyAudio stream
    stream.stop_stream()
    stream.close()

    # Terminate the PyAudio object
    p.terminate()

    # Save the recorded data as a WAV file
    wf = wave.open(filename, "wb")
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b"".join(frames))
    wf.close()

    #print(f"Recording saved as {filename}")

    # Use SpeechRecognition to recognize the speech
    r = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio_data = r.record(source)

    try:
        print("Recognizing...")
        query = r.recognize_google(audio_data, language='en-in')
        print(f"User said: {query}\n")
        return query
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
        return "None"
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return "None"
    except Exception as e:
        print(e)
        print("Unable to Recognize your voice.") 
        return "None"




In [15]:
def preprocess_test(filename, scaler): 
  #load the file
  data, sampling_rate = librosa.load(filename, offset = 0.5)
  #preprocess
  n_fft =  int (sampling_rate * 0.025)
  mfcc = np.mean(librosa.feature.mfcc(y = data, sr = sampling_rate, n_mfcc = 40, n_fft = n_fft).T, axis=0)
  #standardize  
  mfcc_standardized = scaler.transform(mfcc.reshape(1, -1)).reshape(1, -1)
  return mfcc_standardized

In [16]:
# configuration.py

class Configuration:
    OPENAI_API_KEY = "ADD YOUR KEY"


In [17]:

openai.api_key = Configuration.OPENAI_API_KEY
engine =pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

In [18]:
#print("OpenAI Version:", openai.__version__)


In [19]:
def extract_mfcc_features_from_audio(audio_data, sr, num_mfcc=40, n_fft=2048, hop_length=512):
    # Extract MFCC features
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
    # Pad or truncate the features to ensure consistent shape
    if mfccs.shape[1] < 200:
        mfccs = np.pad(mfccs, ((0, 0), (0, 200 - mfccs.shape[1])), mode='constant')
    else:
        mfccs = mfccs[:, :200]
    return mfccs

## ChatGPT Function

In [20]:

def is_long_enough(text, min_word_count=1):
    return len(text.split()) >= min_word_count

def interact_with_ChatGPT(content=messages,drowsy='Notdrowsy',situation=''):
    global messages
    messages.append({"role": "user", "content": content})
    importlib.reload(pyttsx3)
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[1].id)
    engine.say("Hello"+name+situation+" If you wish to end the conversation, just say 'exit' or 'thank you'.")
    engine.runAndWait()

    while True:
        
        importlib.reload(pyttsx3)
        engine = pyttsx3.init()
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[1].id)
        #engine.say("Hello "+name+"! I am Ghost, I am here to keep you awake. If you wish to end the conversation, just say 'exit' or 'thank you'.")
        user_text = extract_text_from_voice()
        if (user_text=='None'):
            messages.append({"role": "system", "content": 'system was not able to detect the user voice. ask the user to speak again'})
            engine.say('Sorry we were not able to detect your voice.')
        if is_long_enough(user_text):
            messages.append({"role": "user", "content": user_text})
            
            query = user_text.lower()   
            
            # All the commands said by user will be 
            # stored here in 'query' and will be
            # converted to lower case for easily 
            # recognition of command
            if 'exit' in query or "thank you" in user_text:
                engine.say("Thanks for giving me your time")
                engine.runAndWait()
                print("Thanks for giving me your time")
                break
                
                
            elif 'wikipedia' in query:
                print('hereeee')
                engine.say('Searching Wikipedia...')
                query = query.replace("wikipedia", "")
                results = wikipedia.summary(query, sentences = 3)
                engine.say("According to Wikipedia")
                print(results)
                engine.say(results)
                messages.append({"role": "system", "content": 'system opened wikipedia ask user if he wants anything else if not user just has to say thank you or exit and ask him to leave the phone and focus on the road'})

            elif 'open youtube' in query:
                engine.say("Here you go to Youtube\n")
                print("hena youtube")
                webbrowser.open("https://youtube.com", new=2)
                messages.append({"role": "system", "content": 'system opened youtube ask user if he wants anything else if not user just has to say thank you or exit and ask him to leave the phone and focus on the road'})


            elif 'open google' in query:
                engine.say("Here you go to Google\n")
                webbrowser.open("https://google.com", new=2)
                messages.append({"role": "system", "content": 'system opened google ask user if he wants anything else if not user just has to say thank you or exit and ask him to leave the phone and focus on the road'})


            elif 'the time' in query:
                strTime = datetime.datetime.now().strftime("% H:% M:% S") 
                engine.say(f"Sir, the time is {strTime}")

            if (drowsy == 'Drowsy' and user_text != 'None'):
                new_audio_file = "C:/Users/lenovo/Desktop/Graduation_Project/output.wav"
                preprocessed_data_test = preprocess_test(new_audio_file, drowsiness_scaler)
                preprocessed_data_test = preprocessed_data_test.reshape((preprocessed_data_test.shape[0], 1, preprocessed_data_test.shape[1]))
                # 3. Make Predictions
                predictions_test = drowsiness_voice_model.predict(preprocessed_data_test,verbose=0)
                # 4. Postprocess audio file => binary
                predicted_class_test = np.argmax(predictions_test)
                if predicted_class_test == 0:
                    result = 'Normal'
                else:
                    result = 'Sleepy'
                    
                if(result == 'Normal'):
                    print("Awake")
                else:
                    print("Driver appears drowsy. Taking action...")
                    messages.append({"role": "system", "content": 'system has detected that the user is drowsy from the camera and now we detected that the user voice is drowsy so we will ask the user to talk and try to be awake and talk with him about his interests and tell him his drowsiness situation and if he was distracted'})
                # pass
                
            # Extend the conversation context by sending the entire conversation history
            response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)
            
            reply = response.choices[0].message.content
            engine.say(reply)
            print(reply)
            engine.runAndWait()
            
            messages.append({"role": "assistant", "content": reply})
            
        else:
            print("Please provide a longer response for a more engaging conversation.")
        
        # Introduce a delay between requests to avoid rate limit issues
        time.sleep(3)



## Real-Time Detection with Cameras for drowsiness and distraction 

In [21]:
# def detect_laptop() :
#     global drow_predicted_class
#     global drowsiness_flag
#     global check_flag_drow
#     #class_names = ["Drowsy","Notdrowsy"]

#     # 0 laptop
#     cv2.startWindowThread() 
#     cap = cv2.VideoCapture(2)

#     if not cap.isOpened():
#         print("Error: Could not open camera.")
#         exit()
    
#     # Load Haar cascade for eye detection
#     eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
#     #print(eye_cascade)
#     # Counter for saving images
#     image_counter = 0
#     drowsiness_output=[]

#     cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
#     cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             print("Error: Could not read frame.")
#             break
#         # Convert the frame to grayscale for eye detection
#         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#         #print(gray)  #print images pixels
#         # Detect eyes in the grayscale frame
#         eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
#         #print(eyes)
#         # Iterate over detected eyes
#         for (ex, ey, ew, eh) in eyes:
#             #print("gowa eye")
#             # Extract the region of interest (eye)
#             eye = frame[ey:ey+eh, ex:ex+ew]
#             # Save the detected eye as an image
#             #eye_filename = f"eye_{image_counter}.jpg"
#             #cv2.imwrite(eye_filename, eye)
#             #print(f"Saved {eye_filename}")
            
#             # Increment image counter
#             image_counter += 1
#             prediction=drowsiness_detection(eye) #change it to drowsiness
#             # Display prediction on the frame
#             #print(prediction)
#             if prediction[0][0] < 0.50:
#                 predicted_class= "Drowsy"
#                 #cv2.putText(frame, 'Closed', (ex, ey-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
#             else:
#                 predicted_class= "Notdrowsy"
#                 # cv2.putText(frame, 'Open', (ex, ey-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
#             drowsiness_output.append(predicted_class)
#             if len(drowsiness_output)==30:
            
#                 most_common = mode(drowsiness_output)
#                 drow_predicted_class=most_common
#                 drowsiness_flag=1
#                 print("300 frames from drowsiness are taken from drowsiness")
#                 cv2.putText(frame, most_common, (ex, ey-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
#                 # Draw a rectangle around the detected eye
#                 cv2.rectangle(frame, (ex, ey), (ex+ew, ey+eh), (255, 0, 0), 2)
#             if (check_flag_drow==1):
#                 drowsiness_output=[]
#                 check_flag_drow=0
#             break

#         cv2.imshow("drowsiness", frame)
#         if cv2.waitKey(1) == 27 :
#             break
#         #if keyboard.is_pressed("s"):
#         #    break


#     cap.release() 
#     cv2.destroyWindow("drowsiness")
 


In [22]:
def detect_laptop() :
    global drow_predicted_class
    global drowsiness_flag
    global check_flag_drow
    
    cv2.startWindowThread() 
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open camera DROW.")
        exit()

    drowsiness_output=[]
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    while True:
        #print("here")
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame drowsiness")
            break
        #print("here1")
        image_input = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # load face detection model
        mp_face = mp.solutions.face_detection.FaceDetection(
            model_selection=1,  # model selection
            min_detection_confidence=0.5  # confidence threshold
        )
        #print("mp_face",mp_face)
        results = mp_face.process(image_input)
        #print("results",results)
        if results.detections is not None:
            #print("results.detections is not None")
            image_rows, image_cols, _ = frame.shape
            mp_face_detection = mp.solutions.face_detection
            mp_drawing = mp.solutions.drawing_utils

            detection = results.detections[0]

            eye_left=mp_face_detection.get_key_point(detection, mp_face_detection.FaceKeyPoint.LEFT_EYE)
            #
            eye_right=mp_face_detection.get_key_point(detection, mp_face_detection.FaceKeyPoint.RIGHT_EYE)



            eye_r = _normalized_to_pixel_coordinates(eye_right.x,eye_right.y, image_cols,image_rows)
            eye_l = _normalized_to_pixel_coordinates(eye_left.x,eye_left.y, image_cols,image_rows)
            
            # Define the size of the rectangle to be drawn
            # Define the size of the rectangle to be drawn based on the eye size
            rectangle_size = int(np.linalg.norm(np.array(eye_r) - np.array(eye_l)) / 2.7)

            # Define the rectangles for each eye
            rectangles = [(eye_r[0]-rectangle_size, eye_r[1]-rectangle_size, eye_r[0]+rectangle_size, eye_r[1]+rectangle_size),
                        (eye_l[0]-rectangle_size, eye_l[1]-rectangle_size, eye_l[0]+rectangle_size, eye_l[1]+rectangle_size)]
            #print("rectangles",rectangles)
            for rect in rectangles:
                x1, y1, x2, y2 = rect
                # Extract the region of interest (eye)
                eye = frame[y1:y2, x1:x2]
                #print("eye",eye)
                if eye.shape[0]>0 and eye.shape[1]>0:
                    #print("da5al al if")
                    # Resize the eye to match the input shape of the model
                    prediction=drowsiness_detection(eye) #change it to drowsiness
                    #print("prediction",prediction)
                    confidence=int(prediction[0][0]*100)  #print in streamlit not the code
                    #print("confidence",confidence)
                    # Display prediction on the frame
                    if prediction[0][0] < 0.60:
                        predicted_class= "Drowsy"
                    else:
                        predicted_class= "Notdrowsy"
                    # Draw a rectangle around the detected eye
                    drowsiness_output.append(predicted_class)
                    if len(drowsiness_output)==30:
                        most_common = mode(drowsiness_output)
                        drow_predicted_class=most_common
                        drowsiness_flag=1
                        print("frames from drowsiness are taken from drowsiness")
                        cv2.putText(frame, str(confidence), (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
                        # Draw a rectangle around the detected eye
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    if (check_flag_drow==1):
                        drowsiness_output=[]
                        check_flag_drow=0
                    break
                else:
                    print(eye.shape)
                    

            

            cv2.imshow("drowsiness", frame)
            if cv2.waitKey(1) == 27 :
                break
            #if keyboard.is_pressed("s"):
            #    break


    cap.release() 
    cv2.destroyWindow("drowsiness")


In [23]:

def detect_phone() :
    global dist_predicted_class
    global distraction_flag
    global check_flag_dist
    class_names = ["Drive safe","Text Right","Talk Right","Text Left","Talk Left","Adjust Radio","Drink","Reach Behind","Makeup & Hair","Talk Passenger"]
    # 1 phone
    cv2.startWindowThread() 
    cap = cv2.VideoCapture(1)

    if not cap.isOpened():
        print("Error: Could not open camera DIST.")
        exit()

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    distracted_output=[]
    while True:
        #weighted average 
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame DIST.")
            break
        RBG = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        resized_image,predicted_class=distraction_detection(RBG,class_names)
        distracted_output.append(predicted_class)
        if len(distracted_output)==30:
            most_common=mode(distracted_output)
            cv2.putText(resized_image, f"Class: {most_common}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            dist_predicted_class= most_common
            print("frames from distraction are taken from distraction")
            distraction_flag=1
        if (check_flag_dist==1): #this means that the check is done so restart 
            distracted_output=[]
            check_flag_dist=0
            #dist_predicted_class= "Drink"
        cv2.imshow("distraction", frame)
        if cv2.waitKey(1) == ord('s'):
            break
        #if keyboard.is_pressed("s"):
        #    break

    cap.release() 
    cv2.destroyWindow("distraction")



In [24]:

def detect_phone_lstm() :
    global dist_predicted_class
    global distraction_flag
    global check_flag_dist
    class_names = ["Drive safe","Text Right","Talk Right","Text Left","Talk Left","Adjust Radio","Drink","Reach Behind","Makeup & Hair","Talk Passenger"]
    frames=[]
    # 1 phone
    cv2.startWindowThread() 
    cap = cv2.VideoCapture(2)

    if not cap.isOpened():
        print("Error: Could not open camera DIST.")
        exit()

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    distracted_output=[]
    while True:
        #weighted average 
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame DIST.")
            break
        RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(RGB)
        if len(frames)==4:
            #print("4 Frames for one prediction of LSTM")
            predicted_class=distraction_lstm_detection(frames,class_names)
            distracted_output.append(predicted_class)
            if len(distracted_output)==30:
                most_common=mode(distracted_output)
                cv2.putText(frame, f"Class: {most_common}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                dist_predicted_class= most_common
                print("frames from distraction are taken from distraction")
                distraction_flag=1
            if (check_flag_dist==1): #this means that the check is done so restart 
                distracted_output=[]
                check_flag_dist=0
                #dist_predicted_class= "Drink"
            frames=[]
            
        cv2.imshow("distraction", frame)
        if cv2.waitKey(1) == ord('s'):
            break
        #if keyboard.is_pressed("s"):
        #    break

    cap.release() 
    cv2.destroyWindow("distraction")



## Threading between the 2 Cameras

In [25]:
#thread1=threading.Thread(target=detect_phone) # thread for camera phone 
thread1=threading.Thread(target=detect_phone_lstm)

In [26]:
thread3=threading.Thread(target=detect_laptop) # thread for camera laptop

## Merging Cases Function

In [27]:
def check_result(): #check both drowsiness and distraction at the same time 
    global distraction_flag
    global drowsiness_flag
    global check_flag_drow
    global check_flag_dist
    while True:
        importlib.reload(pyttsx3)
        engine = pyttsx3.init()
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[1].id)
        if (drowsiness_flag==1 and distraction_flag==1):
            #print(f"thread 1 / mobile camera {thread1.is_alive()}")
            #print(f"thread 3 / laptop camera {thread3.is_alive()}")

            if (thread1.is_alive()==False) or (thread3.is_alive()==False):
                
                #print("hbreak men al checK 3lshan wa7da men cameras esc 5ALST ")
                break
            #print("gowa al if")
            
            predicted_Distraction_vision_class_name=dist_predicted_class
            predicted_Drowsiness_vision_class_name=drow_predicted_class

            #FOR REVIWING
            #predicted_Distraction_vision_class_name="Talk Passenger"
            #predicted_Drowsiness_vision_class_name="Drowsy"


            if (predicted_Distraction_vision_class_name == "Drive safe") and (predicted_Drowsiness_vision_class_name == "Notdrowsy"):
                print("Pass because he is driving safely and not drowsy as well")
            
            elif (predicted_Distraction_vision_class_name == "Drive safe") and (predicted_Drowsiness_vision_class_name == "Drowsy"):
                print("driver is driving safe but he is drowsy so we need chatbot in order to confirm so")
            
                print("Chatgpt in progress")
                interact_with_ChatGPT(situation = 'You are being Drowsy, Lets have a conversation to keep you awake',drowsy=predicted_Drowsiness_vision_class_name,content="ALERT the user: It seems like user is feeling drowsy.keep responses very short around one sentence please. It's essential to stay alert while driving. Consider taking a short break, opening the window for fresh air, or having a caffeinated drink if it's safe to do so. Your safety is our priority.I am Ghost, your chatbot companion. As your driving companion, my primary goal is to ensure your safety and well-being on the road.")
            
            elif (predicted_Distraction_vision_class_name in ["Talk Right", "Talk Left"] and (predicted_Drowsiness_vision_class_name == "Drowsy")):
                print("Chatgpt in progress")

                interact_with_ChatGPT(situation = 'You are being distracted and drowsy. Please focus on the road and be alert', drowsy=predicted_Drowsiness_vision_class_name,content="I am Ghost, your chatbot companion.keep responses very short around one sentence please. As your driving companion, my primary goal is to ensure your safety and well-being on the road. Please keep your responses short to avoid distractions. If you ever need assistance, feel free to ask. ALERT: It appears that you might be feeling drowsy and distracted by something else. It's crucial to maintain focus and alertness while driving. Firstly, try to address the distraction by safely pulling over if necessary or eliminating the source of distraction. Once that's addressed, consider taking a short break to rest and refresh yourself. If possible, engage in some simple exercises or stretches to increase alertness. Your safety and the safety of others on the road are our top priorities. You can end the conversation at any time by saying thank you or exit")
            elif (predicted_Distraction_vision_class_name in ["Talk Right", "Talk Left"] and (predicted_Drowsiness_vision_class_name == "Notdrowsy")):
                print("driver is distracted by Talkling and also drowsy so we need chatbot")  # to be removed later
            
                engine.say("PLEASE FOCUS ON THE ROAD YOU HAVE BEEN TALKING ON THE PHONRE FOR A WHILE")
                engine.runAndWait()
            elif (predicted_Distraction_vision_class_name in ["Adjust Radio"] ):
                print("driver is distracted by Adjust Radio")  # to be removed later
                engine.say("PLEASE FOCUS ON THE ROAD YOU HAVE BEEN ADJUSTING RADIO FOR A WHILE")
                engine.runAndWait()

            elif (predicted_Distraction_vision_class_name in ["Drink"] ):
                print("driver is distracted by  Drink")  # to be removed later
                engine.say("PLEASE FOCUS ON THE ROAD YOU HAVE BEEN DRINKING FOR A WHILE")
                engine.runAndWait()

            elif (predicted_Distraction_vision_class_name in ["Reach Behind"] ):
                print("driver is distracted by Reach Behind")  # to be removed later
                engine.say("PLEASE FOCUS ON THE ROAD YOU HAVE BEEN REACHING BEHIND FOR A WHILE")
                engine.runAndWait()

            elif (predicted_Distraction_vision_class_name in ["Makeup & Hair"] ):
                print("driver is distracted by Makeup & Hair")  # to be removed later
                engine.say("PLEASE FOCUS ON THE ROAD YOU ARE HAVE BEEN FIXING YOUR HAIR OR MAKEUP FOR A WHILE")
                engine.runAndWait()

            elif (predicted_Distraction_vision_class_name in ["Text Right", "Text Left"]):
                print("driver is distracted by texting")
                interact_with_ChatGPT(situation = 'You are using your phone while driving. I can open to you google, youtube or search wikipedia for you is there is anything that I can help you with' ,content = 'User is Texting while driving  and you are ghost a chatbot. keep responses very short around one sentence please to help the user stay focused while driving so ask the user to stay focused and if he want to open google or youtube or search wikipedia and never say that you can not open youtube as our system can and will open it so that the user can focus more ron the road if the user does not need anything tell him to tell you thank you or exit')
            elif (predicted_Distraction_vision_class_name in ["Talk Passenger"]) :
                model = Speaking_voice_model
                audio_file_path = TakeRealTimeAudio()
                #print("audio submit")
                #audio_file_path = real.result()
                predicted_Speaking_class_name = detect_speaking_file(model, audio_file_path, target_sr=target_sample_rate, gain_factor=2.0)
                if predicted_Speaking_class_name == "Speaking":
                    engine.say("YOU ARE SPEAKING TO A PASSANGER PLEASE FOCUS ON THE ROAD")
                    engine.runAndWait()
                elif predicted_Speaking_class_name == "Silent":
                    print("silent AND NOT DROWSY SO pass")
           
            distraction_flag=0
            drowsiness_flag=0
            check_flag_dist=1
            check_flag_drow=1
            print("the system will wait for some frames and 30 seconds  till the next detection")
            time. sleep(30) #get the frame every 10 seconds
        



In [28]:
thread1.start()
thread3.start()

#print(dist_predicted_class)
# In order for the cameras to capture few frames and calculate the weighted average 
#time. sleep(30) 
thread2=threading.Thread(target=check_result)
thread2.start()


c:\Users\lenovo\miniconda3\envs\gp\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


frames from drowsiness are taken from drowsiness
frames from distraction are taken from distraction
driver is driving safe but he is drowsy so we need chatbot in order to confirm so
Chatgpt in progress
Listening ...
Stopped
Recognizing...
User said: no thank you I'm fine I'm awake

Thanks for giving me your time
the system will wait for some frames and 30 seconds  till the next detection
frames from drowsiness are taken from drowsiness
frames from distraction are taken from distraction
driver is distracted by texting
Listening ...
Stopped
Recognizing...
User said: search Google for me please yes please open Google Now

I have opened Google for you. Is there anything specific you would like me to search for you? And remember, please keep your focus on the road for your safety.
Listening ...
Stopped
Recognizing...
User said: no thank you that's perfect thank you

Thanks for giving me your time
the system will wait for some frames and 30 seconds  till the next detection
frames from drowsi

c:\Users\lenovo\miniconda3\envs\gp\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\lenovo\miniconda3\envs\gp\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\lenovo\miniconda3\envs\gp\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\lenovo\min

Speaking Model in progress...
the system will wait for some frames and 30 seconds  till the next detection
frames from distraction are taken from distraction
frames from drowsiness are taken from drowsiness
(0, 0, 3)
(0, 0, 3)
(0, 0, 3)
(0, 0, 3)
driver is distracted by Reach Behind
the system will wait for some frames and 30 seconds  till the next detection
frames from drowsiness are taken from drowsiness
frames from distraction are taken from distraction
driver is distracted by  Drink
the system will wait for some frames and 30 seconds  till the next detection
Error: Could not read frame DIST.Error: Could not read frame drowsiness

